In [1]:
%display latex
from pylie import *
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace


In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    assert e2.is_monic()
    def _order (der):
        if der != 1:
            return pylie.order_of_derivative(der)
        else :
            return [0]*len(context._independent)
    def func(e):    
        try:
            return e.operator().function()
        except AttributeError:
            return e.operator()      
    print ("--------------------------------------- entering reduce")    
    #set_trace()
    ld = e2.Lder()
    e2_order = _order (ld)    
    while True:
        for t in e1._p:
            print ("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
            reduction_found = False
            d = t._d
            c = t._coeff
            t.show()
            ld.show()
            if func(ld) != func(d):
                break
            e1_order = _order (d)
            dif = []
            for a, b in zip (e1_order, e2_order):
                dif += [a - b]
            if sum(e1_order) == 3:
                set_trace()
            if all (map (lambda h: h == 0, dif)) :
                ds = set([_._d for _ in e1._p +  e2._p])
                e3 = e1._orig - c * e2._orig
                for _ds in ds:
                    e3 = e3.collect(_ds)
                e1 = Differential_Polynomial(e3, context)
                if len(e1._p) > len(ds)                :
                    set_trace ()
                reduction_found = True
                break
            if all (map (lambda h: h >= 0, dif)):
                variables_to_diff = []
                for i in range (len(context._independent)):
                    if dif [i] != 0:
                        variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))
                ds = set([_._d for _ in e1._p + e2._p] +  [diff(_._d , *variables_to_diff) for _ in e2._p])
                #set_trace()
                e3 = e1._orig - c * diff(e2._orig, *variables_to_diff)
                for _ds in ds:
                    e3 = e3.collect(_ds) 
                e1 =  Differential_Polynomial (e3, context)
                if len(e1._p) > len(ds)                :
                    set_trace ()
                reduction_found = True
                break
            print ("no reduction")
        if not reduction_found:
            return e1        

In [3]:
#indeterminates = (x, y) = var("x y")
#R=PolynomialRing(QQ, indeterminates)
#z  = function ("z")(*indeterminates)
#ctx = Context((z,), indeterminates)
#print("*"*70)
#e1 = diff(z,y)-x**2/y**2*diff(z,x)-(x-y)*z(x,y)/y**2
#e1_dp = Differential_Polynomial (e1, ctx)
#print (e1_dp)
#print("*"*70)
#f1 = diff(z, x) + z(x,y)/x
#f1_dp = Differential_Polynomial (f1, ctx)
#f1_dp.is_monic()
#print (f1_dp)
#print("*"*70)
#f2 = diff(z, y) + z(x,y)/y
#f2_dp = Differential_Polynomial (f2, ctx)
#print (f2_dp)
#r1 = reduce (e1_dp, f1_dp, ctx)
#r1
#r1._orig
#r2 = reduce (r1, f2_dp, ctx)

In [4]:
def Lfunc(e):
    return e

In [5]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [6]:
def Lcm(e):
    return 0

In [7]:
def CompleteSystem(S, context):
    return S

In [8]:
def FindIntegrableConditions(S, context):
    return S

In [9]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [10]:

def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res

In [11]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 = Differential_Polynomial(diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y, ctx)
f1._orig

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [12]:
f2=Differential_Polynomial(diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x,ctx)
f2._orig

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [13]:
f3 = Differential_Polynomial(diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2 ,ctx)
f3._orig

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [14]:
f4 = Differential_Polynomial(diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2), ctx)
f4._orig
f4._p

[<pylie.JanetBasis.DTerm object at 0x7f438cacbbb0>,
 <pylie.JanetBasis.DTerm object at 0x7f438769f940>,
 <pylie.JanetBasis.DTerm object at 0x7f4386d12bb0>,
 <pylie.JanetBasis.DTerm object at 0x7f438770aeb0>,
 <pylie.JanetBasis.DTerm object at 0x7f438cad9e80>,
 <pylie.JanetBasis.DTerm object at 0x7f4386d12e50>]

In [15]:
f5 = Differential_Polynomial(diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2), ctx)
f5._orig

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [16]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        #print ("While")
        #p._orig.show()  
        r = S[i+1:]
        if not r:
            S = Reorder(S, context,True)
            return S[:]    
        print ("=====================================================", len(r))
        have_reduced = False
        checkidx = i+1
        for _r in r:
            #print ("Try to reduce {} with {}".format(S[checkidx], S[i].Lder()))
            res = reduce (_r, p, context)
            if res._orig != _r._orig:
                #print ("have reduced!")
                #res._orig.show()
                #for gurki in res._p:
                #    print ((gurki._coeff *gurki._d).collect (gurki._d))
                have_reduced = True                
            S[checkidx] = res
            checkidx += 1
        print("This is the new System before reorder")   
        for _ in S:          
            _.show()
        S = Reorder(S, context, ascending = True)
        print("This is the new System........")
        for _ in S:          
            _.show()
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            i = 0
            

In [17]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    for _ in S:
        _.show()
    print ("....................................")
    S = Reorder(S, context, ascending = True)
    for _ in S:
        _.show()
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [ ]:
checkS=JanetBasis([f3._orig,f1._orig,f5._orig, f2._orig,f4._orig], ctx)

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

1/2*x*diff(z(x, y), y)/(x^2*y + y^2) - w(x, y)/y + diff(w(x, y), y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

...

4*x^5*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 4*x^6*y^2*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 2*x^4*y^2*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^6*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^7*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*x^3*y^3*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*x^4*y^3*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - x^5*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x*y^3*diff(w(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 8*x^2*y^3*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 4*x*y^3*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 14*x^4*y*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 14*x^5*y*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^4*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 24*x*y^4*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 24*x^2*y^4*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + x^2*y*diff(w(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 7*x^3*y*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 12*y^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 6*y^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + 2*x^2*y*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 12*y^4*diff(z(x, y), x)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + x^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 2*x^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 2*x^6*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 30*x^2*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 30*x^3*y^2*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 8*x^2*y^3*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 16*y^5*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*y^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x) - 1/4*(8*x^4*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 4*x^5/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 32*x^2*y^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 28*x^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 48*y^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 4*x^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y^2) - 60*x*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 32*y^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^2) + 4/x + 18*x^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 52*y^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x) + 8*y^5/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^4) + 24/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - x^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y^3) - 2/y^2 + 8*y/x^3 - 16*y^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^3) - 3*x/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y^2) - 8/(x^2*y) + 10*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^2) + 4*y^2/x^5 - 6/x^4 + 1/(x*y^3) - 2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x*y) + 2/(x^3*y^2))*w(x, y) + x^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 6*y^2*diff(w(x, y), y)/(2*x

ipdb>  e1._orig.expand().collect(diff(z,y)).show()


4*x^4*y^3*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 2*x^4*y^2*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^5*y*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^5*y^2*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 16*x^2*y^4*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^4*y*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - x^5*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x*y^3*diff(w(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 8*x^2*y^3*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 4*x*y^3*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + x^6*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) - 2*x^2*y^2*diff(z(x, y), x, y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 14*x^3*y^2*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 8*x^3*y^3*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 24*y^5*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + x^5*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 8*x^2*y^2*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + x^2*y*diff(w(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 7*x^3*y*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 12*y^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 6*y^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + 2*x^2*y*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 12*y^4*diff(z(x, y), x)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + x^3*diff(z(x, y), x, y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 2*x^4*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 7*x^4*y*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) - 6*y^3*diff(z(x, y), x, y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 30*x*y^3*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 12*x*y^4*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 16*y^6*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^2) + 7*x^3*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 12*y^3*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + x^4*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 6*y^2*diff(w(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 15*x*y^2*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 6*y^5*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^3) + 8*y^5*diff(w(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^2) + 12*y^2*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 12*y^5*diff(z(x, y), x)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^3) - x^5*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)*y) + 6*x*y*diff(z(x, y), x, y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 9*x^2*y*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 15*x^2*y^2*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) - 6*y^4*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^2) - 26*y^4*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x) - 8*y^5*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)*x) + 4*y^7*diff(z(x, y), x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^4) + 1/8*(32*x^5*y^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 32*x^6*y^2/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 16*x^6/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 16*x

ipdb>  e1._orig.expand().collect(diff(z,x,y)).collect(diff(w,y)).show()


4*x^5*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 4*x^6*y^2*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^6*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^7*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*x^3*y^3*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*x^4*y^3*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^4*y*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 4*x*y^3*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 14*x^4*y*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 14*x^5*y*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 2*x^4*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 24*x*y^4*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 24*x^2*y^4*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + x^5*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 8*x^2*y^2*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 2*x^2*y*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 12*y^4*diff(z(x, y), x)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + x^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 2*x^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 2*x^6*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 30*x^2*y^2*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 30*x^3*y^2*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 8*x^2*y^3*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*(x^2 + y)) + 16*y^5*diff(z(x, y), y)/((x^4 + 2*x^2*y + y^2)*(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 16*y^5*diff(z(x, y), y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x) + 7*x^3*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 12*y^3*w(x, y)/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 1/4*(8*x^4*y^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 4*x^5/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 8*x*y^3/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 32*x^2*y^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 4*x^2*y/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 28*x^3*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 48*y^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 24*y^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + 4*x^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) + 24*y^2/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 60*x*y^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 24*y^5/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^3) + 32*y^5/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^2) - 4*x/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + 18*x^2/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) + 4*y/x + 36*y^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^2) - 52*y^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x) - 8*y^6/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^5) + 8*y^6/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^4) - x^3/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y^2) - 2/y + 24*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)) - 14*y/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x) + 8*y^2/x^3 + 16*y^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*x^4) - 16*y^4/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*x^3) - 8/x^2 + 1/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*y) - 3*x/((2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y)*(2*x^2*y + 2*y^2 - x)*y) - 10*y^2/((2*x^2*y^2 - x^3 + 2*y^3

In [ ]:
for i in (f1._orig, f2._orig,f3._orig, f4._orig,f5._orig):
    i.show()

In [ ]:
for _ in checkS:
    _._orig.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a

In [ ]:
f1._orig.show()

In [ ]:
f2._orig.show()

In [ ]:
f3._orig.show()

In [ ]:
f4._orig.show()

In [ ]:
 f5._orig.show()

In [ ]:
f5a=(f5._orig - (-1/y)*f1._orig).expand().simplify(); f5a.show()

In [ ]:
f2a=(f2._orig - (y/x)* f1._orig).expand().collect(diff(z,y)); f2a.show()

In [ ]:
(f4._orig - diff(f1._orig, x)).expand().collect(diff(z, y)).show()